Things to accomplish:
1. Drop useless columns (location is redundant, collision ID)
2. use latitude and logitude values to retrieve borough, zipcode and on street name values
3. drop nan rows for NUMBER OF PERSONS INJURED/KILLED columns (very low amount)
4. Try to simplify factor vehicle and type code column values
5. Create classes as suggested by team (see whatsapp chat)

Minor details:
1. Lowercasing data
2. Make sure street names, boroughs and zipcode match same format

Latitude and Longitude issues:
1. We got NaN values (make sure existing data remains)
2. we got 0, 0 coordinates (make sure we arent receiving false data from false coordinates)
We can make sure we have lat and long coordinates in the range of the boroughs

In [1]:
!pip install geopy

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import datetime

In [3]:
# Define the path to the folder
folder_path = Path("C:/Users/crazy/OneDrive - The City College of New York/DSE I2100 - Applied Machine Learning and Data Mining/Project")
csv_file = folder_path.glob("*.csv").__next__()

# Load CSV file into DataFrame
df = pd.read_csv(csv_file)


C:\Users\crazy\AppData\Local\Temp\ipykernel_20080\545171645.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [4]:
df.head() # don't need location (redundant), or collision ID

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2070069 entries, 0 to 2070068
Data columns (total 29 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   CRASH DATE                     2070069 non-null  object 
 1   CRASH TIME                     2070069 non-null  object 
 2   BOROUGH                        1426009 non-null  object 
 3   ZIP CODE                       1425759 non-null  object 
 4   LATITUDE                       1836747 non-null  float64
 5   LONGITUDE                      1836747 non-null  float64
 6   LOCATION                       1836747 non-null  object 
 7   ON STREET NAME                 1631008 non-null  object 
 8   CROSS STREET NAME              1288388 non-null  object 
 9   OFF STREET NAME                346688 non-null   object 
 10  NUMBER OF PERSONS INJURED      2070051 non-null  float64
 11  NUMBER OF PERSONS KILLED       2070038 non-null  float64
 12  NUMBER OF PEDE

## The Following Code verifies that values for 'LATITUDE', 'LONGITUDE', 'LOCATION' columns are always present together:

In [6]:
# Verification that LATITUDE, LONGITUDE and LOCATION exist together

# Get the indices where LATITUDE is missing, but LOCATION is not
missing_lat_indices = df[(df['LATITUDE'].isnull()) & (~df['LOCATION'].isnull())].index

# Get the indices where LONGITUDE is missing, but LOCATION is not
missing_lon_indices = df[(df['LONGITUDE'].isnull()) & (~df['LOCATION'].isnull())].index

# Get the indices where LOCATION is missing, but LATITUDE is not
missing_loc_lat_indices = df[(df['LATITUDE'].notnull()) & (df['LOCATION'].isnull())].index

# Get the indices where LOCATION is missing, but LONGITUDE is not
missing_loc_lon_indices = df[(df['LONGITUDE'].notnull()) & (df['LOCATION'].isnull())].index

# Get the indices where LATITUDE is missing, but LONGITUDE is not
missing_lat_lon_indices = df[(df['LATITUDE'].isnull()) & (df['LONGITUDE'].notnull())].index

# Get the indices where LONGITUDE is missing, but LATITUDE is not
missing_lon_lat_indices = df[(df['LONGITUDE'].isnull()) & (df['LATITUDE'].notnull())].index

# Print results
print("Indices with missing LATITUDE but present LOCATION:")
print(df.loc[missing_lat_indices, ['LATITUDE', 'LONGITUDE', 'LOCATION']])

print("Indices with missing LONGITUDE but present LOCATION:")
print(df.loc[missing_lon_indices, ['LATITUDE', 'LONGITUDE', 'LOCATION']])

print("Indices with missing LOCATION but present LATITUDE:")
print(df.loc[missing_loc_lat_indices, ['LATITUDE', 'LONGITUDE', 'LOCATION']])

print("Indices with missing LOCATION but present LONGITUDE:")
print(df.loc[missing_loc_lon_indices, ['LATITUDE', 'LONGITUDE', 'LOCATION']])

print("Indices with missing LATITUDE but present LONGITUDE:")
print(df.loc[missing_lat_lon_indices, ['LATITUDE', 'LONGITUDE', 'LOCATION']])

print("Indices with missing LONGITUDE but present LATITUDE:")
print(df.loc[missing_lon_lat_indices, ['LATITUDE', 'LONGITUDE', 'LOCATION']])


Indices with missing LATITUDE but present LOCATION:
Empty DataFrame
Columns: [LATITUDE, LONGITUDE, LOCATION]
Index: []
Indices with missing LONGITUDE but present LOCATION:
Empty DataFrame
Columns: [LATITUDE, LONGITUDE, LOCATION]
Index: []
Indices with missing LOCATION but present LATITUDE:
Empty DataFrame
Columns: [LATITUDE, LONGITUDE, LOCATION]
Index: []
Indices with missing LOCATION but present LONGITUDE:
Empty DataFrame
Columns: [LATITUDE, LONGITUDE, LOCATION]
Index: []
Indices with missing LATITUDE but present LONGITUDE:
Empty DataFrame
Columns: [LATITUDE, LONGITUDE, LOCATION]
Index: []
Indices with missing LONGITUDE but present LATITUDE:
Empty DataFrame
Columns: [LATITUDE, LONGITUDE, LOCATION]
Index: []


## Creating test dataframes to maintain original dataframe:

In [7]:
test_df = df.copy()

# Dropping useless columns
test_df.drop(columns=['LOCATION', 'COLLISION_ID'], inplace=True)

## The Following code replaces 0 value rows with NaN for 'LATITUDE' & 'LONGITUDE' columns:

In [8]:
# verify we have valid coordinates in the data

# Filter the DataFrame for latitude and longitude values less than 1 and greater than -1
filtered_df = test_df[(test_df['LATITUDE'] < 1) & (test_df['LATITUDE'] > -1) | (test_df['LONGITUDE'] < 1) & (test_df['LONGITUDE'] > -1)]
print(filtered_df[['LATITUDE','LONGITUDE']])

         LATITUDE  LONGITUDE
44            0.0        0.0
47            0.0        0.0
212           0.0        0.0
252           0.0        0.0
537           0.0        0.0
...           ...        ...
2066947       0.0        0.0
2067378       0.0        0.0
2069047       0.0        0.0
2069552       0.0        0.0
2070021       0.0        0.0

[4350 rows x 2 columns]


In [9]:
# Approximate coordinates for New York City:
# Maximum Latitude: 40.9176 (Northernmost point of the Bronx)
# Minimum Latitude: 40.4774 (Southernmost point of Staten Island)
# Maximum Longitude: -73.7004 (Easternmost point of Queens)
# Minimum Longitude: -74.2591 (Westernmost point of Staten Island)

# Define the maximum and minimum values for latitude and longitude
max_latitude = 40.9176
min_latitude = 40.4774
max_longitude = -73.7004
min_longitude = -74.2591

# Filter the DataFrame based on the conditions for latitude and longitude
invalid_latitudes = (test_df['LATITUDE'] > max_latitude) | (test_df['LATITUDE'] < min_latitude)
invalid_longitudes = (test_df['LONGITUDE'] > max_longitude) | (test_df['LONGITUDE'] < min_longitude)

# Replace the values with NaN where the conditions are not met
test_df.loc[invalid_latitudes, 'LATITUDE'] = np.nan
test_df.loc[invalid_longitudes, 'LONGITUDE'] = np.nan

In [10]:
test_filtered_df = test_df[(test_df['LATITUDE'] < 1) & (test_df['LATITUDE'] > -1) | (test_df['LONGITUDE'] < 1) & (test_df['LONGITUDE'] > -1)]
print(test_filtered_df[['LATITUDE','LONGITUDE']])

Empty DataFrame
Columns: [LATITUDE, LONGITUDE]
Index: []


In [11]:
test_df2 = test_df.head(50).copy()
test_df3 = test_df.copy()

## The following code creates a function to replace NaN values in the 'BOROUGH', 'ZIP CODE', 'ON STREET NAME' with actual values:

In [12]:
# Function to find values for BOROUGH, ZIP CODE & ON STREET NAME using coordinates
geolocator = Nominatim(user_agent="accident_severity_prediction")

def reverse_geocode(lat, lon, borough, zipcode, street_name):
    
    if np.isnan(lat) or np.isnan(lon) or bool(borough) == bool(zipcode) == bool(street_name) == "True":
        #return [None, None, None]
        return [np.nan, np.nan, np.nan]
                                                                                                 
    else:
        
        try:
            location = geolocator.reverse((lat, lon))
            address = location.raw
            borough = address.get('address', {}).get('suburb')
            zipcode = address.get('address', {}).get('postcode')
            on_street_name = address.get('address', {}).get('road')
            return [borough, zipcode, on_street_name]

        except:
            #return [None, None, None]
            return [np.nan, np.nan, np.nan]

In [13]:
# Apply the reverse_geocode function to each row of the dataframe
new_values = test_df2.apply(lambda row: reverse_geocode(row['LATITUDE'], row['LONGITUDE'], row['BOROUGH'], row['ZIP CODE'], row['ON STREET NAME']), axis=1, result_type='expand')

# Replace NaN values in 'borough', 'zipcode', and 'address' columns with the corresponding values from the reverse_geocode function
test_df2['BOROUGH'].fillna(new_values[0], inplace=True)
test_df2['ZIP CODE'].fillna(new_values[1], inplace=True)
test_df2['ON STREET NAME'].fillna(new_values[2], inplace=True)

In [14]:
test_df2[['LATITUDE','LONGITUDE','BOROUGH','ZIP CODE','ON STREET NAME']].head(50)
#print(new_values)

,LATITUDE,LONGITUDE,BOROUGH,ZIP CODE,ON STREET NAME
0,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY
1,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER
2,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE
3,40.667202,-73.866500,BROOKLYN,11208.0,Loring Avenue
4,40.683304,-73.917274,BROOKLYN,11233.0,SARATOGA AVENUE
5,NaN,NaN,NaN,NaN,MAJOR DEEGAN EXPRESSWAY RAMP
6,40.709183,-73.956825,Brooklyn,11249,BROOKLYN QUEENS EXPRESSWAY
7,40.868160,-73.831480,BRONX,10475.0,Baychester Avenue
8,40.671720,-73.897100,BROOKLYN,11207.0,Pitkin Avenue
9,40.751440,-73.973970,MANHATTAN,10017.0,3 AVENUE


The function does not remove any existing values, only adds values to the 'BOROUGH', 'ZIP CODE', 'ON STREET NAME' columns.

In [ ]:
# Verifying if function works for huge dataset

# Apply the reverse_geocode function to each row of the dataframe
new_values = test_df.apply(lambda row: reverse_geocode(row['LATITUDE'], row['LONGITUDE']), axis=1, result_type='expand')

# Replace NaN values in 'borough', 'zipcode', and 'address' columns with the corresponding values from the reverse_geocode function
test_df['BOROUGH'].fillna(new_values[0], inplace=True)
test_df['ZIP CODE'].fillna(new_values[1], inplace=True)
test_df['ON STREET NAME'].fillna(new_values[2], inplace=True)

Issue with trying to run big dataset with function, maybe add code to prevent iteration when borough, zipcode and onstreetname are not NaN

1. Also, need to complete and make sure new values match dataset, like borough data
2. convert zipcode data to float64
3. convert borough and street name to str object

## Renaming duplicate values for "BOROUGH" column:

In [15]:
# Get the counts of unique values in the "BOROUGH" column
borough_counts = test_df2['BOROUGH'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'BOROUGH' column:")
print(borough_counts)

# Calculate the total sum of the counts
total_sum = borough_counts.sum()
print("Total sum of counts:", total_sum)


Unique values and their counts in the 'BOROUGH' column:
BROOKLYN         11
BRONX             6
QUEENS            5
Queens County     4
Manhattan         4
Brooklyn          2
MANHATTAN         2
STATEN ISLAND     1
Staten Island     1
The Bronx         1
Name: BOROUGH, dtype: int64
Total sum of counts: 37


In [16]:
# Convert all values in the "BOROUGH" column to lowercase
test_df2['BOROUGH'] = test_df2['BOROUGH'].str.lower()

# Replace specific values in the "BOROUGH" column
test_df2['BOROUGH'].replace({'the bronx': 'bronx', 'queens county': 'queens'}, inplace=True)

In [17]:
# Get the counts of unique values in the "BOROUGH" column
borough_counts = test_df2['BOROUGH'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'BOROUGH' column:")
print(borough_counts)

# Calculate the total sum of the counts
total_sum = borough_counts.sum()
print("Total sum of counts:", total_sum)

Unique values and their counts in the 'BOROUGH' column:
brooklyn         13
queens            9
bronx             7
manhattan         6
staten island     2
Name: BOROUGH, dtype: int64
Total sum of counts: 37


## Converting zipcodes to string objects

In [18]:
# Get the counts of unique values in the "BOROUGH" column
zipcode_counts = test_df2['ZIP CODE'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(zipcode_counts)

# Calculate the total sum of the counts
total_sum = zipcode_counts.sum()
print("Total sum of counts:", total_sum)

Unique values and their counts in the 'ZIP CODE' column:
11217.0    2
11208.0    1
11230.0    1
11211.0    1
10455.0    1
11385.0    1
10128      1
11225.0    1
11379      1
10314      1
10462      1
10029      1
11220.0    1
11411.0    1
10452.0    1
10466.0    1
10033      1
11215.0    1
10301.0    1
11233.0    1
11214      1
11249      1
10475.0    1
11207.0    1
10017.0    1
11385      1
11413.0    1
11434.0    1
10037      1
11226.0    1
10463.0    1
11354      1
10001.0    1
11364      1
11372.0    1
10453.0    1
Name: ZIP CODE, dtype: int64
Total sum of counts: 37


In [20]:
# Convert 'ZIP CODE' column to float, handling NaN values
test_df2['ZIP CODE'] = pd.to_numeric(test_df2['ZIP CODE'], errors='coerce')

# Convert 'ZIP CODE' column to int (this will automatically drop any decimal points)
test_df2['ZIP CODE'] = test_df2['ZIP CODE'].astype(float).astype('Int64')  # Convert to Int64 to handle NaN and preserve integer dtype

# Convert 'ZIP CODE' column back to string
test_df2['ZIP CODE'] = test_df2['ZIP CODE'].astype(str)

In [21]:
# Get the counts of unique values in the "BOROUGH" column
zipcode_counts = test_df2['ZIP CODE'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(zipcode_counts)

# Calculate the total sum of the counts
total_sum = zipcode_counts.sum()
print("Total sum of counts:", total_sum)

Unique values and their counts in the 'ZIP CODE' column:
<NA>     13
11385     2
11217     2
10314     1
11215     1
11211     1
10455     1
10128     1
11225     1
11379     1
10462     1
10301     1
10029     1
11220     1
11411     1
10452     1
10466     1
10033     1
11230     1
10037     1
11208     1
11372     1
11364     1
10001     1
11354     1
10463     1
11226     1
11434     1
11214     1
11413     1
10017     1
11207     1
10475     1
11249     1
11233     1
10453     1
Name: ZIP CODE, dtype: int64
Total sum of counts: 50


## Converting street names to lowercase string objects

In [22]:
# Get the counts of unique values in the "BOROUGH" column
street_counts = test_df2['ON STREET NAME'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(street_counts)

# Calculate the total sum of the counts
total_sum = street_counts.sum()
print("Total sum of counts:", total_sum)

Unique values and their counts in the 'ZIP CODE' column:
WHITESTONE EXPRESSWAY               2
THROGS NECK BRIDGE                  2
Empire Boulevard                    1
FULTON STREET                       1
GRAND STREET                        1
Eagle Avenue                        1
67th Avenue                         1
EAST 93 STREET                      1
EASTCHESTER ROAD                    1
KINGSLAND AVENUE                    1
WILLIAMSBURG BRIDGE OUTER ROADWA    1
HUTCHINSON RIVER PARKWAY            1
ELIOT AVENUE                        1
EAST 18 STREET                      1
STATEN ISLAND EXPRESSWAY            1
BOSTON ROAD                         1
EAST 107 STREET                     1
4th Avenue                          1
BOSCOBEL PLACE                      1
BRONX RIVER PARKWAY                 1
EAST 162 STREET                     1
CROSS BRONX EXPY                    1
12th Street                         1
VICTORY BOULEVARD                   1
QUEENSBORO BRIDGE UPPER        

In [23]:
# Convert all values in the "BOROUGH" column to lowercase
test_df2['ON STREET NAME'] = test_df2['ON STREET NAME'].str.lower()


In [24]:
# Get the counts of unique values in the "BOROUGH" column
street_counts = test_df2['ON STREET NAME'].value_counts()

# Print unique values and their counts
print("Unique values and their counts in the 'ZIP CODE' column:")
print(street_counts)

# Calculate the total sum of the counts
total_sum = street_counts.sum()
print("Total sum of counts:", total_sum)

Unique values and their counts in the 'ZIP CODE' column:
whitestone expressway               2
throgs neck bridge                  2
empire boulevard                    1
fulton street                       1
grand street                        1
eagle avenue                        1
67th avenue                         1
east 93 street                      1
eastchester road                    1
kingsland avenue                    1
williamsburg bridge outer roadwa    1
hutchinson river parkway            1
eliot avenue                        1
east 18 street                      1
staten island expressway            1
boston road                         1
east 107 street                     1
4th avenue                          1
boscobel place                      1
bronx river parkway                 1
east 162 street                     1
cross bronx expy                    1
12th street                         1
victory boulevard                   1
queensboro bridge upper        